# Keras Regression Model

Importing libraries.

In [49]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
# loading the data
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')

## Data Exploration.

In [3]:
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [4]:
concrete_data.shape

(1030, 9)

In [5]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [6]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

Our data appears to have no missing data.

In [11]:
# defining features and target
features = concrete_data[concrete_data.columns[concrete_data.columns != 'Strength']]
target = concrete_data['Strength']

In [36]:
# defining loop for 50 iterations of splitting and training.
def loop50(model, X, y, epochs):
    mse_list = []
    for n in range(1,50):
        features_train, features_test, target_train, target_test = train_test_split(X, y, test_size=0.3)
        model.fit(features_train, target_train, epochs = epochs, verbose = 0)
        mse_list.append(mean_squared_error(target_test, model.predict(features_test)))
    mse_array = np.asarray(mse_list)
    return mse_array

In [53]:
# defining report function, accepts np.array output from 50 iteration loop, returns list of MSEs their mean and standard deviation.
def report(mse_array):
    mse_mean =np.mean(mse_array)
    mse_std = np.std(mse_array)
    print('Mean Squared Errors:', mse_array, 'Mean of MSEs:', mse_mean, 'Standard Deviation of MSEs', mse_std)

## Part A: Baseline Model (No-normalization, single hidden layer, 50 epochs)

In [60]:
def baseline_model():
    # defining layers
    model= Sequential()
    model.add(Dense(10, activation = 'relu', input_shape =(features.shape[1],)))
    model.add(Dense(1))
    # compiling model
    model.compile(optimizer = 'adam', loss= 'mean_squared_error')
    return model

In [61]:
model_A = baseline_model()

In [62]:
model_A_mse_array = loop50(model_A, features, target, 50)

In [63]:
report(model_A_mse_array)

Mean Squared Errors: [121.6425067  117.48503401 123.18673434 115.82852943 100.31800545
 101.71791939 104.8049602  104.13231736 112.98467007 112.49227191
 109.68986815 104.43255735 116.78177839 115.38468041 119.98719961
 118.99343001 108.01934371 114.94589022 110.8063217  113.80794696
 108.19111366 109.79921109 115.81805454 111.20440287 100.3266877
 126.60627434 107.29668419 114.90058498 126.97010438 116.88936625
  99.70140207 138.62216504 102.77248718 122.15899151 100.58962911
 109.24669873 109.95016802 123.12870763  98.34707444 108.29098412
 123.90952232 109.31322398 108.19982069 104.86854683 107.37561091
 118.42049211 110.64529705 112.06946045 113.32212462] Mean of MSEs: 112.37503788141832 Standard Deviation of MSEs 8.218307021506172


## Part B: Normalized Data

In [64]:
model_B = baseline_model()

In [65]:
features_norm = (features - features.mean()) / features.std()

In [66]:
model_B_mse_array = loop50(model_B, features_norm, target, 50)

In [67]:
report(model_B_mse_array)

Mean Squared Errors: [296.59324973 172.33243275 111.34676708 102.09782696  80.67008038
  79.73795664  60.2363238   61.2230308   45.63479995  51.02163376
  40.46632196  44.23964996  33.94086169  37.93960283  44.85681412
  38.21564444  39.94349382  41.31351565  41.76453095  34.03242463
  32.83796887  37.45989452  32.38646717  34.85009566  37.25925165
  34.00461271  40.25916214  41.15989479  36.60810783  43.16516977
  36.06272569  33.54553083  34.35073173  41.35334661  30.99668721
  35.08933703  40.82736777  34.0169982   33.29076772  36.52544595
  38.28879071  35.66809929  42.07185752  34.78434411  33.80581019
  39.64283338  33.79444485  33.85290896  37.85273991] Mean of MSEs: 51.29425213662161 Standard Deviation of MSEs 43.11625987020986


The normalization reduces Mean Squared Error significantly, thus also reducing our mean of mean squared errors. Standard Deviation is higher, but largely due to the more significant MSE reduction over the first 10 of 50 iterations.

## Part C: 100 Epochs

In [68]:
model_C = baseline_model()

In [69]:
model_C_mse_array = loop50(model_C, features_norm, target, 100)

In [70]:
report(model_C_mse_array)

Mean Squared Errors: [168.33554568  72.04496543  55.38751711  46.44880874  45.53624955
  41.31995973  40.39524055  36.46420009  38.31444606  33.32117129
  36.00727604  33.98636631  40.86831065  37.64594159  38.03842258
  37.97572235  42.36448858  43.02335661  36.0520977   34.98261271
  42.15409986  39.44607296  36.27090447  34.3250308   36.1555989
  30.55884986  32.18670858  34.38954507  33.15880101  38.3167352
  37.96627435  32.13103142  30.9253539   35.55850022  31.70242254
  32.16752125  33.40712315  42.74934582  38.46342517  35.43615229
  31.4455142   30.63178417  33.39627489  36.35979366  34.6959875
  33.85158793  38.06854016  34.14518072  36.08806991] Mean of MSEs: 40.29928427199504 Standard Deviation of MSEs 19.687316925663243


Additional epochs mean the algorithm had twice as much training with each iteration thus improving early results, reducing the Mean of MSEs and the standard deviation compared to version C. 

## Part D: More Hidden Layers

In [73]:
def deep_model():
    # defining layers
    model= Sequential()
    model.add(Dense(10, activation = 'relu', input_shape =(features.shape[1],)))
    model.add(Dense(10, activation = 'relu'))
    model.add(Dense(10, activation = 'relu'))
    model.add(Dense(1))
    # compiling model
    model.compile(optimizer = 'adam', loss= 'mean_squared_error')
    return model

In [74]:
model_D = deep_model()

In [75]:
model_D_mse_array = loop50(model_D, features_norm, target, 100)

In [76]:
report(model_D_mse_array)

Mean Squared Errors: [82.77762687 42.82125087 31.19370211 28.98130114 27.66348503 29.6044826
 25.33339692 25.16010686 30.26296226 25.51546368 21.25158945 22.80493461
 22.03023296 20.17112698 19.30174486 18.97285503 22.90051325 22.94394713
 16.00902531 18.43318387 15.76967059 19.3615235  18.54577234 22.05639813
 19.33082605 22.37116997 20.24121885 19.8715072  17.39895986 18.81971755
 18.94393445 18.87343668 17.56088244 16.03962551 17.75536651 19.57456176
 21.84074892 16.93954752 15.70436588 15.8892529  19.1219251  20.32899454
 17.89231544 15.22692628 16.88588706 17.52644678 18.27063421 15.30232175
 16.78699933] Mean of MSEs: 22.129874874489342 Standard Deviation of MSEs 10.135311501636991


The deeper model produces significantly better results, reducing MSE across the board with lower standard deviation compared to model C.  